In [1]:
import os
import sys

CARTOLA_HOME = os.path.abspath('..')

print(CARTOLA_HOME)

if not os.path.isdir(CARTOLA_HOME):
    raise Exception('Cartola project is missing!')

sys.path.append(CARTOLA_HOME)

/home/cavani/Workspace/cartola-game


In [ ]:
!pip install pyomo

In [3]:
from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

In [4]:
opt = SolverFactory('cbc')

In [5]:
model = AbstractModel()
model.n = Param(default=4)
model.x = Var(RangeSet(model.n), within=Binary)
def o_rule(model):
    return summation(model.x)
model.o = Objective(rule=o_rule)
model.c = ConstraintList()

In [6]:
instance = model.create_instance()
results = opt.solve(instance)
instance.display()

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :     0 :     1 : False : False : Binary
          2 :     0 :     0 :     1 : False : False : Binary
          3 :     0 :     0 :     1 : False : False : Binary
          4 :     0 :     0 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :     0

  Constraints:
    c : Size=0
        Key : Lower : Body : Upper


In [7]:
instance.x

In [8]:
for i in range(5):
    instance.solutions.load_from(results)

    expr = 0
    for j in instance.x:
        if instance.x[j].value == 0:
            expr += instance.x[j]
        else:
            expr += (1-instance.x[j])
    instance.c.add( expr >= 1 )

    results = opt.solve(instance)
    print ("\n===== iteration",i)
    instance.display()


===== iteration 0
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :     0 :     1 : False : False : Binary
          2 :     0 :     0 :     1 : False : False : Binary
          3 :     0 :     0 :     1 : False : False : Binary
          4 :     0 :     1 :     1 : False : False : Binary

  Objectives:
    o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :     1

  Constraints:
    c : Size=1
        Key : Lower : Body : Upper
          1 :   1.0 :    1 :  None

===== iteration 1
Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :     0 :     1 : False : False : Binary
          2 :     0 :     0 :     1 : False : False : Binary
          3 :     0 :     1 :     1 : False : False : Binary
          4 :     0 :     0 :     1 : False : False : Binary

  Objectives:
    o

In [9]:
v = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}

limit = 14

model = ConcreteModel()

model.ITEMS = Set(initialize=v.keys())

model.x = Var(model.ITEMS, within=Binary)

model.value = Objective(expr=sum(v[i]*model.x[i] for i in model.ITEMS), sense=maximize)

model.weight = Constraint(expr=sum(w[i]*model.x[i] for i in model.ITEMS) <= limit)

results = opt.solve(model)
model.display()

Model unknown

  Variables:
    x : Size=4, Index=ITEMS
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :     1 :     1 : False : False : Binary
        screwdriver :     0 :     1 :     1 : False : False : Binary
              towel :     0 :     1 :     1 : False : False : Binary
             wrench :     0 :     0 :     1 : False : False : Binary

  Objectives:
    value : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :    25

  Constraints:
    weight : Size=1
        Key  : Lower : Body : Upper
        None :  None :   12 :  14.0


In [10]:
from cartola_game.model import Team, Formation
from cartola_game.data import load_season
from cartola_game.engine import GameEngine, GameInstance

In [11]:
season_2015 = load_season('2015')
x = GameEngine(season_2015)
i = GameInstance(x)
data = i.data

In [12]:
players = list(data.players_data.keys())
n_players = len(players)
print('Players:', n_players)

Players: 1041


In [13]:
opt_values = [p.price for p in data.players_data.values()]
print('Opt (prices): {}, ..., {}'.format(str(opt_values[:3])[:-1], str(opt_values[-4:-1])[1:]))

Opt (prices): [8.67, 7.99, 1.0, ..., 6.0, 11.39, 11.07]


In [14]:
budget = i.total_money
print('Budget:', budget)

Budget: 100.0


In [15]:
prices = [p.price for p in data.players_data.values()]
print('Prices: {}, ..., {}'.format(str(prices[:3])[:-1], str(prices[-4:-1])[1:]))

Prices: [8.67, 7.99, 1.0, ..., 6.0, 11.39, 11.07]


In [16]:
formation = Formation.F433
formation

<Formation.F433: ('4-3-3', [PositionSlot(position=<Position.GOALKEEPER: 'Goleiro'>, size=1), PositionSlot(position=<Position.WINGER: 'Lateral'>, size=2), PositionSlot(position=<Position.DEFENDER: 'Zagueiro'>, size=2), PositionSlot(position=<Position.MIDFIELDER: 'Meia'>, size=3), PositionSlot(position=<Position.FORWARD: 'Atacante'>, size=3), PositionSlot(position=<Position.COACH: 'Técnico'>, size=1)])>

In [17]:
model = ConcreteModel()

model.PLAYERS = RangeSet(0,n_players-1)

model.x = Var(model.PLAYERS, within=Binary)

model.objective = Objective(expr=sum(opt_values[i] * model.x[i] for i in model.PLAYERS), sense=maximize)

model.budget = Constraint(expr=sum(prices[i] * model.x[i] for i in model.PLAYERS) <= budget)

model.team_size = Constraint(expr=sum(model.x[i] for i in model.PLAYERS) == 12)

for pos, n in formation.positions:
    players_position = [i for i, p in enumerate(players) if p.position == pos]
    model.add_component(pos.name, Constraint(expr=sum(model.x[i] for i in players_position) == n))

results = opt.solve(model)

if results.solver.status != SolverStatus.ok \
    or results.solver.termination_condition != TerminationCondition.optimal:
    raise Exception('No optimal solution!\n\n' + str(results.solver))
    
model.display()

Model unknown

  Variables:
    x : Size=1041, Index=PLAYERS
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
           0 :     0 :     0 :     1 : False : False : Binary
           1 :     0 :     0 :     1 : False : False : Binary
           2 :     0 :     0 :     1 : False : False : Binary
           3 :     0 :     0 :     1 : False : False : Binary
           4 :     0 :     0 :     1 : False : False : Binary
           5 :     0 :     0 :     1 : False : False : Binary
           6 :     0 :     0 :     1 : False : False : Binary
           7 :     0 :     0 :     1 : False : False : Binary
           8 :     0 :     0 :     1 : False : False : Binary
           9 :     0 :     0 :     1 : False : False : Binary
          10 :     0 :     0 :     1 : False : False : Binary
          11 :     0 :     0 :     1 : False : False : Binary
          12 :     0 :     0 :     1 : False : False : Binary
          13 :     0 :     0 :     1 : False : False : Binary
         

In [18]:
model.objective()

100.00000000000001

In [19]:
selected_players = list()
for i in model.x:
    if model.x[i].value:
        selected_players.append(players[i])

team = Team(formation, selected_players) 

print(team)
print('\nTeam price: {:.2f}'.format(data.price(team.players)))
print('Team score: {:.2f}'.format(x.score_engine(1).score(team.players)))

4-3-3

Goleiro -> Jucemar de Oliveira Cordeiro Júnior
Lateral -> Everton José Modesto Silva
Lateral -> Sueliton Pereira de Aguiar
Zagueiro -> Matheus Ferraz Pereira
Zagueiro -> Matheus Simonete Bressanelli
Meia -> Josef de Souza Dias
Meia -> Marcos Guilherme de Almeida S. Matos
Meia -> Renato Adriano Jacó de Morais
Atacante -> Allano Brendon de Souza Lima
Atacante -> Deyvison Rogério da Silva
Atacante -> Ángel Rodrigo Romero Villamayor
Técnico -> Argélico Fucks

Team price: 100.00
Team score: 39.17


In [20]:
opt = SolverFactory('cbc')

def milp_team(players, opt_values, prices, budget, formation, verbose=False):
    model = ConcreteModel()
    
    model.PLAYERS = RangeSet(0,len(players)-1)
    model.x = Var(model.PLAYERS, within=Binary)

    model.objective = Objective(expr=sum(opt_values[i] * model.x[i] for i in model.PLAYERS), sense=maximize)

    model.budget = Constraint(expr=sum(prices[i] * model.x[i] for i in model.PLAYERS) <= budget)

    N = 0
    for pos, n in formation.positions:
        players_position = [i for i, p in enumerate(players) if p.position == pos]
        model.add_component(pos.name, Constraint(expr=sum(model.x[i] for i in players_position) == n))
        N += n
    model.team_size = Constraint(expr=sum(model.x[i] for i in model.PLAYERS) == N)

    results = opt.solve(model)
    if results.solver.status != SolverStatus.ok \
        or results.solver.termination_condition != TerminationCondition.optimal:
        raise Exception('No optimal solution!\n\n' + str(results.solver))

    if verbose:
        model.display()
    
    opt_value = model.objective()

    selected_players = list()
    for i in model.x:
        if model.x[i].value:
            selected_players.append(players[i])
    
    return Team(formation, selected_players), opt_value

team, price = milp_team(players, opt_values, prices, budget, formation)

print(team)
print('\nTeam price:', price)

4-3-3

Goleiro -> Rodolfo Pereira de Castro
Lateral -> Sidcley Ferreira Pereira
Lateral -> Sueliton Pereira de Aguiar
Zagueiro -> Matheus Ferraz Pereira
Zagueiro -> Rafael Dumas
Meia -> Josef de Souza Dias
Meia -> Marcos Guilherme de Almeida S. Matos
Meia -> Renato Adriano Jacó de Morais
Atacante -> Eduardo Pereira Rodrigues
Atacante -> Marcos do Sul Bezerra
Atacante -> Renato Kayser de Souza
Técnico -> Argélico Fucks

Team price: 100.00000000000001
